# Tamas Flesch Thesis - EDA step

## LMJU - UpGrad - DS

### Fifa 23 Ultimate Team player price prediction based on the player's attributes

<span style="color:red">*UPDATE*</span>

The document was updated with the latest EA FC 24 dataset to be able to compare the results with the previous FIFA 23 dataset.

Table of contents
1. Read data
2. Clean data
    - Price_Variation
        - convert the field to numeric
    - Height (extract height in cm into a new column)
        - a few rows were droped because of missing data
    - BodyType (extrct 2 new column, body type text and weight in kg)
        - a few rows were droped because of missing data
    - Club (create new column for ICON players 1/0)
    - League
        - kept the 5 major european league, MLS and 2 special league (ICONS, World cup)
    - Nation
        - kept the 4 country with the most player count which almost have 1000+ players
    - Card_Version
        - removed the column, has mixed values
    - Position columns
        - created 3 separate columns for the alternate positions (those could be NaN if there are no alt positions)
3. Categorical dummy columns
    - create dummy category columns for the normal categorical columns
    - handle alt position columns, these are have missing values as well
    - UPDATE FOR EA FC 24: new column, gender
4. Export the final dataset
5. visualization TODO!!!
    - outliers check
    - correlation matrix
    - https://towardsdatascience.com/an-extensive-guide-to-exploratory-data-analysis-ddd99a03199e

### 1. Read data

#### Imports

In [1]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Pandas and NumPy
import pandas as pd, numpy as np

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
# Importing all datasets
futbin_data = pd.read_csv("eafc24_players_2024-06-07.csv")
futbin_data.head()

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Price_Variation,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats,Gender
0,Erling Haaland,Manchester City,Norway,Premier League,97,ST,CF,TOTY,Controlled Lengthy,1980000.0,0.25%,4,5,H,M,96.0,98,80,91,56,97,"195cm | 6'5""",Unique (94kg),1645,518,2481,Male
1,Kylian Mbappé,Paris SG,France,Ligue 1 Uber Eats,97,ST,"CF,LW",TOTY,Controlled Explosive,6250000.0,1.63%,5,4,H,L,99.0,96,88,97,47,88,"182cm | 6'0""",Unique (75kg),1651,515,2465,Male
2,Lionel Messi,Inter Miami CF,Argentina,MLS,97,CF,"CAM,RW,ST",TOTY,Mostly Explosive,2830000.0,1.33%,5,4,M,L,91.0,96,97,98,40,77,"169cm | 5'7""",Messi (67kg),-4941,499,2430,Male
3,Pelé,EA FC ICONS,Brazil,Icons,97,CAM,"CF,ST",FUT Birthday Icon,Mostly Explosive,6400000.0,6.68%,5,5,H,M,95.0,96,92,96,60,81,"173cm | 5'8""",Unique (70kg),272,520,2532,Male
4,Johan Cruyff,EA FC ICONS,Holland,Icons,97,CF,ST,Golazo Icon,Mostly Explosive,3550000.0,1.40%,5,5,H,L,93.0,95,93,97,50,80,"180cm | 5'11""",Avg & Lean (71kg),159,508,2460,Male


#### check and understand the Price data column

In [5]:
futbin_data.Price.describe()

count    2.258100e+04
mean     1.737879e+04
std      1.692791e+05
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      3.000000e+02
max      8.200000e+06
Name: Price, dtype: float64

In [6]:
len(futbin_data[futbin_data.Price == 0])

13564

In [7]:
futbin_data[futbin_data.Price == 0].head()

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Price_Variation,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats,Gender
9,Jeremie Frimpong,Leverkusen,Holland,Bundesliga,97,RWB,"RB,RM",Champion TOTS,Mostly Explosive,0.0,0,5,4,H,H,99.0,84,93,97,91,88,"171cm | 5'7""",Avg & Normal (64kg),32,552,2573,Male
10,Harry Kane,FC Bayern München,England,Bundesliga,97,ST,CF,Champion TOTS,Controlled,0.0,0,4,5,H,H,90.0,99,93,93,56,91,"188cm | 6'2""",Unique (89kg),34,522,2547,Male
11,Erling Haaland,Manchester City,Norway,Premier League,97,ST,CF,Champion TOTS,Controlled Lengthy,0.0,0,4,5,H,M,97.0,99,80,92,57,97,"195cm | 6'5""",Unique (94kg),20,522,2505,Male
12,Virgil van Dijk,Liverpool,Holland,Premier League,97,CB,0,Champion TOTS,Lengthy,0.0,0,3,3,M,H,92.0,71,84,85,97,94,"193cm | 6'4""",Unique (92kg),37,523,2462,Male
14,Kylian Mbappé,Paris SG,France,Ligue 1 Uber Eats,97,ST,"CF,LW",Champion TOTS,Controlled Explosive,0.0,0,5,4,H,L,99.0,97,90,98,45,90,"182cm | 6'0""",Unique (75kg),-389,519,2486,Male


SBC concept, ingame, there is a mini game, where players can finish squad building challenges (SBCs), and get valuable player. These players are not in the market, so players can't buy them on the market, that's why they havn't got a price. So I'll remove these cards from the dataset.

In [8]:
futbin_data = futbin_data[futbin_data.Price > 0]

In [9]:
len(futbin_data[futbin_data.Price == 0])

0

### 2. Clean data

We need only numeric data, so first of all I'll need to investigate the columns, extract the numeric fields, like the height or the body weight. 

Then create dummy columns for the categorical columns like the run style. 

At the last step, I'll create some derived columns where I need to merge some information from columns with a lot of different text values. Like the club or the card version. These columns have a lot of different, distinct values, so I will select some of the most valueable categories and will create a special coulmn with 1/0 values. 1 for the most special categories and 0 for the other columns.

#### Price_Variation

In [10]:
futbin_data.Price_Variation.value_counts()

0          4940
14.29%      362
50.00%      130
37.50%      115
25.00%       85
12.50%       70
8.33%        69
3.92%        68
28.57%       67
2.94%        62
4.90%        56
75.00%       53
5.88%        53
4.76%        50
5.00%        48
7.14%        48
10.00%       43
87.50%       42
62.50%       41
6.25%        35
7.69%        31
15.38%       30
1.92%        27
1.41%        27
5.56%        26
4.55%        26
16.67%       25
11.11%       24
2.91%        24
6.67%        24
51.00%       23
2.88%        23
100.00%      21
26.67%       20
88.89%       20
0.96%        18
5.19%        17
57.69%       17
36.36%       17
1.52%        17
1.94%        17
3.45%        16
22.22%       16
3.85%        16
7.84%        15
23.53%       15
9.09%        15
4.85%        15
0.93%        15
55.56%       14
13.33%       14
3.88%        13
10.53%       13
40.00%       12
18.18%       12
5.71%        12
3.57%        12
33.33%       12
1.64%        12
5.26%        12
2.78%        12
20.00%       11
0.76%   

too much missing valuees, and based on the differences, it seems to me that this column has a lot of outliners as well, I think it is better to remove it from the dataset.

In [11]:
futbin_data = futbin_data.drop(['Price_Variation'], 1)

#### Height

In [12]:
futbin_data.head()["Height"][0]

'195cm | 6\'5"'

In [13]:
futbin_data.Height.info()

<class 'pandas.core.series.Series'>
Int64Index: 9017 entries, 0 to 22580
Series name: Height
Non-Null Count  Dtype 
--------------  ----- 
9017 non-null   object
dtypes: object(1)
memory usage: 140.9+ KB


In [14]:
futbin_data.Height.value_counts()

185cm | 6'1"     652
188cm | 6'2"     626
190cm | 6'3"     464
187cm | 6'2"     460
180cm | 5'11"    422
186cm | 6'1"     421
183cm | 6'0"     403
191cm | 6'3"     365
178cm | 5'10"    348
189cm | 6'2"     347
184cm | 6'0"     326
175cm | 5'9"     308
193cm | 6'4"     266
182cm | 6'0"     261
192cm | 6'4"     257
173cm | 5'8"     242
181cm | 5'11"    211
170cm | 5'7"     202
176cm | 5'9"     201
177cm | 5'10"    196
194cm | 6'4"     178
168cm | 5'6"     174
172cm | 5'8"     173
174cm | 5'9"     165
179cm | 5'10"    153
195cm | 6'5"     151
171cm | 5'7"     137
196cm | 6'5"     125
169cm | 5'7"     104
163cm | 5'4"     101
165cm | 5'5"      78
167cm | 5'6"      69
164cm | 5'5"      64
197cm | 6'6"      52
161cm | 5'3"      44
160cm | 5'3"      40
198cm | 6'6"      40
166cm | 5'5"      35
162cm | 5'4"      30
199cm | 6'6"      21
0                 20
158cm | 5'2"      16
201cm | 6'7"      14
155cm | 5'1"      11
157cm | 5'2"      10
200cm | 6'7"       9
202cm | 6'8"       9
159cm | 5'3" 

In [15]:
# remove 0 values, 4 records

In [16]:
futbin_data.shape[0]

9017

In [17]:
futbin_data[futbin_data.Height == "0"]

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats,Gender
131,Charlton,EA FC ICONS,England,Icons,94,CAM,CF,Golazo Icon,Explosive,450000.0,4,5,H,M,92.0,93,90,94,53,77,0,Avg & Normal (72kg),818,499,2442,Male
236,Antunes Coimbra,EA FC ICONS,Brazil,Icons,93,CAM,"CM,CF",Future Stars Icon,Mostly Explosive,1310000.0,5,5,H,M,91.0,94,93,93,70,75,0,Avg & Normal (71kg),597,516,2499,Male
343,Charlton,EA FC ICONS,England,Icons,92,CAM,CF,Icon,Explosive,148000.0,4,5,H,M,90.0,91,87,91,50,74,0,Avg & Normal (72kg),4403,483,2360,Male
351,Antunes Coimbra,EA FC ICONS,Brazil,Icons,92,CAM,"CM,CF",Squad Found. C.L.,Mostly Explosive,636000.0,5,5,H,M,90.0,93,92,92,65,73,0,Avg & Normal (71kg),1174,505,2447,Male
499,Antunes Coimbra,EA FC ICONS,Brazil,Icons,91,CAM,"CM,CF",Icon,Mostly Explosive,195000.0,5,5,H,M,89.0,92,91,91,62,71,0,Avg & Normal (71kg),3538,496,2402,Male
20609,Hamm,EA FC ICONS,United States,Icons,95,ST,"CAM,CF,RW",TOTY Icon,Controlled Explosive,2190000.0,5,5,H,M,94.0,95,88,94,57,83,0,Short & Normal (57kg),861,511,2477,Female
20629,Abily,EA FC ICONS,France,Icons,94,CM,CAM,FUT Birthday Icon,Controlled Explosive,640000.0,5,4,H,H,87.0,93,92,93,88,84,0,Short & Normal (59kg),155,537,2583,Female
20630,Abily,EA FC ICONS,France,Icons,94,CM,CAM,FUT Birthday Icon,Controlled Explosive,700000.0,4,5,H,H,87.0,93,92,93,88,84,0,Short & Normal (59kg),159,537,2583,Female
20631,Prinz,EA FC ICONS,Germany,Icons,94,ST,CF,Golazo Icon,Controlled,70000.0,4,5,H,M,90.0,94,85,88,54,91,0,Avg & Normal (77kg),181,502,2416,Female
20656,Hamm,EA FC ICONS,United States,Icons,93,ST,"CAM,CF,RW",Icon,Controlled Explosive,318000.0,5,5,H,M,92.0,93,86,92,54,81,0,Short & Normal (57kg),-1247,498,2411,Female


In [18]:
futbin_data = futbin_data.drop(futbin_data[futbin_data.Height == "0"].index)

In [19]:
futbin_data.shape[0]

8997

In [20]:
futbin_data[futbin_data.Height == "0"]

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats,Gender


In [21]:
futbin_data['Height_in_cm'] = futbin_data['Height'].str[:3]

In [22]:
futbin_data['Height_in_cm'].info()

<class 'pandas.core.series.Series'>
Int64Index: 8997 entries, 0 to 22580
Series name: Height_in_cm
Non-Null Count  Dtype 
--------------  ----- 
8997 non-null   object
dtypes: object(1)
memory usage: 140.6+ KB


In [23]:
futbin_data['Height_in_cm'] = pd.to_numeric(futbin_data['Height_in_cm'])

In [24]:
futbin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8997 entries, 0 to 22580
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    8997 non-null   object 
 1   Club                    8997 non-null   object 
 2   Nation                  8997 non-null   object 
 3   League                  8997 non-null   object 
 4   Rating                  8997 non-null   int64  
 5   Main_Position           8997 non-null   object 
 6   Alternate_Positions     8997 non-null   object 
 7   Card_Version            8997 non-null   object 
 8   Run_Style               8997 non-null   object 
 9   Price                   8997 non-null   float64
 10  Skills_Star             8997 non-null   int64  
 11  Weak_Foot_Star          8997 non-null   int64  
 12  Attack_Workrate         8997 non-null   object 
 13  Defense_Workrate        8997 non-null   object 
 14  Pace / Diving           8997 non-null  

#### Body type

In [25]:
futbin_data.BodyType.info()

<class 'pandas.core.series.Series'>
Int64Index: 8997 entries, 0 to 22580
Series name: BodyType
Non-Null Count  Dtype 
--------------  ----- 
8997 non-null   object
dtypes: object(1)
memory usage: 140.6+ KB


In [26]:
futbin_data.BodyType.value_counts()

Tall & Normal  (80kg)     305
Tall & Normal  (82kg)     242
Tall & Normal  (85kg)     216
Tall & Normal  (83kg)     211
Tall & Normal  (84kg)     208
Avg & Normal  (70kg)      190
Tall & Normal  (78kg)     184
Tall & Normal  (81kg)     166
Avg & Normal  (75kg)      162
Tall & Lean  (80kg)       152
Tall & Normal  (88kg)     148
Tall & Normal  (86kg)     131
Avg & Normal  (76kg)      129
Tall & Lean  (75kg)       127
Tall & Normal  (79kg)     124
Avg & Normal  (73kg)      123
Avg & Normal  (78kg)      122
Tall & Normal  (75kg)     122
Avg & Normal  (74kg)      115
Tall & Normal  (87kg)     113
Avg & Normal  (72kg)      106
Avg & Lean  (70kg)        105
Avg & Normal  (80kg)      104
Tall & Lean  (78kg)       101
Tall & Normal  (77kg)      99
Avg & Normal  (79kg)       94
Avg & Normal  (77kg)       94
Avg & Lean  (68kg)         91
Short & Normal  (60kg)     91
Tall & Normal  (90kg)      88
Tall & Lean  (77kg)        87
Tall & Normal  (76kg)      85
Avg & Normal  (67kg)       83
Tall & Nor

I need to create two more columns from this one. The first one would be a numeric field in kg. The second is a categorical field with the body type.

In [27]:
futbin_data[['BodyType_Text', 'BodyType_Weight']] = futbin_data['BodyType'].str.split('  ', 1, expand=True)

First try, using '  ' (double spaces), let's try, is it working?

In [28]:
futbin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8997 entries, 0 to 22580
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    8997 non-null   object 
 1   Club                    8997 non-null   object 
 2   Nation                  8997 non-null   object 
 3   League                  8997 non-null   object 
 4   Rating                  8997 non-null   int64  
 5   Main_Position           8997 non-null   object 
 6   Alternate_Positions     8997 non-null   object 
 7   Card_Version            8997 non-null   object 
 8   Run_Style               8997 non-null   object 
 9   Price                   8997 non-null   float64
 10  Skills_Star             8997 non-null   int64  
 11  Weak_Foot_Star          8997 non-null   int64  
 12  Attack_Workrate         8997 non-null   object 
 13  Defense_Workrate        8997 non-null   object 
 14  Pace / Diving           8997 non-null  

Some rows are missing from the Weight coulmn, maybe double-space is not the right separator for every row.

In [29]:
futbin_data.BodyType_Text.value_counts()

Tall & Normal     2885
Avg & Normal      1998
Tall & Lean       1174
Avg & Lean        1065
Short & Normal     699
Unique             417
Avg & Stocky       234
Tall & Stocky      234
Short & Lean       208
Short & Stocky      47
Salah               11
CR7                  5
Neymar               5
Messi                5
Ronaldinho           4
R9                   2
Courtois             2
Shaqiri              2
Name: BodyType_Text, dtype: int64

In [30]:
futbin_data.BodyType_Weight.value_counts()

(80kg)     606
(75kg)     539
(78kg)     496
(70kg)     419
(82kg)     365
(77kg)     346
(76kg)     344
(79kg)     333
(81kg)     327
(83kg)     325
(73kg)     316
(74kg)     316
(84kg)     291
(72kg)     289
(85kg)     286
(68kg)     217
(88kg)     195
(71kg)     195
(67kg)     190
(86kg)     189
(69kg)     184
(65kg)     167
(66kg)     167
(60kg)     160
(87kg)     152
(62kg)     142
(64kg)     138
(90kg)     136
(63kg)     112
(61kg)     110
(58kg)     109
(89kg)     107
(59kg)      86
(57kg)      66
(56kg)      66
(55kg)      64
(91kg)      63
(92kg)      62
(94kg)      56
(93kg)      53
(54kg)      52
(95kg)      26
(53kg)      25
(52kg)      23
(96kg)      20
(51kg)      15
(98kg)      14
(97kg)      10
(99kg)       7
(100kg)      4
(49kg)       4
(102kg)      3
(48kg)       2
(103kg)      2
(104kg)      1
(105kg)      1
(101kg)      1
(47kg)       1
(46kg)       1
(50kg)       1
Name: BodyType_Weight, dtype: int64

Second try, sometimes we have 2 or 3 spaces as a separator so I will try a parenthesis (

In [31]:
futbin_data[['BodyType_Text', 'BodyType_Weight']] = futbin_data['BodyType'].str.split('(', 1, expand=True)

In [32]:
futbin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8997 entries, 0 to 22580
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    8997 non-null   object 
 1   Club                    8997 non-null   object 
 2   Nation                  8997 non-null   object 
 3   League                  8997 non-null   object 
 4   Rating                  8997 non-null   int64  
 5   Main_Position           8997 non-null   object 
 6   Alternate_Positions     8997 non-null   object 
 7   Card_Version            8997 non-null   object 
 8   Run_Style               8997 non-null   object 
 9   Price                   8997 non-null   float64
 10  Skills_Star             8997 non-null   int64  
 11  Weak_Foot_Star          8997 non-null   int64  
 12  Attack_Workrate         8997 non-null   object 
 13  Defense_Workrate        8997 non-null   object 
 14  Pace / Diving           8997 non-null  

In [33]:
futbin_data.BodyType_Text.value_counts()

Tall & Normal       2885
Avg & Normal        1998
Tall & Lean         1174
Avg & Lean          1065
Short & Normal       699
Unique               417
Avg & Stocky         234
Tall & Stocky        234
Short & Lean         208
Short & Stocky        47
Salah                 11
CR7                    5
Neymar                 5
Messi                  5
Ronaldinho             4
R9                     2
Courtois               2
Shaqiri                2
Name: BodyType_Text, dtype: int64

In [34]:
futbin_data.BodyType_Weight.value_counts()

80kg)     606
75kg)     539
78kg)     496
70kg)     419
82kg)     365
77kg)     346
76kg)     344
79kg)     333
81kg)     327
83kg)     325
73kg)     316
74kg)     316
84kg)     291
72kg)     289
85kg)     286
68kg)     217
88kg)     195
71kg)     195
67kg)     190
86kg)     189
69kg)     184
65kg)     167
66kg)     167
60kg)     160
87kg)     152
62kg)     142
64kg)     138
90kg)     136
63kg)     112
61kg)     110
58kg)     109
89kg)     107
59kg)      86
57kg)      66
56kg)      66
55kg)      64
91kg)      63
92kg)      62
94kg)      56
93kg)      53
54kg)      52
95kg)      26
53kg)      25
52kg)      23
96kg)      20
51kg)      15
98kg)      14
97kg)      10
99kg)       7
100kg)      4
49kg)       4
102kg)      3
48kg)       2
103kg)      2
104kg)      1
105kg)      1
101kg)      1
47kg)       1
46kg)       1
50kg)       1
Name: BodyType_Weight, dtype: int64

In [35]:
futbin_data.BodyType_Weight.count()

8997

In [36]:
futbin_data[futbin_data.BodyType_Weight.isnull() == True]

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight


In [37]:
futbin_data = futbin_data.drop(futbin_data[futbin_data.BodyType_Weight.isnull() == True].index)

In [38]:
futbin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8997 entries, 0 to 22580
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    8997 non-null   object 
 1   Club                    8997 non-null   object 
 2   Nation                  8997 non-null   object 
 3   League                  8997 non-null   object 
 4   Rating                  8997 non-null   int64  
 5   Main_Position           8997 non-null   object 
 6   Alternate_Positions     8997 non-null   object 
 7   Card_Version            8997 non-null   object 
 8   Run_Style               8997 non-null   object 
 9   Price                   8997 non-null   float64
 10  Skills_Star             8997 non-null   int64  
 11  Weak_Foot_Star          8997 non-null   int64  
 12  Attack_Workrate         8997 non-null   object 
 13  Defense_Workrate        8997 non-null   object 
 14  Pace / Diving           8997 non-null  

In [39]:
futbin_data.BodyType_Text.value_counts()

Tall & Normal       2885
Avg & Normal        1998
Tall & Lean         1174
Avg & Lean          1065
Short & Normal       699
Unique               417
Avg & Stocky         234
Tall & Stocky        234
Short & Lean         208
Short & Stocky        47
Salah                 11
CR7                    5
Neymar                 5
Messi                  5
Ronaldinho             4
R9                     2
Courtois               2
Shaqiri                2
Name: BodyType_Text, dtype: int64

In [40]:
futbin_data.BodyType_Text = futbin_data.BodyType_Text.str.strip()

In [41]:
futbin_data.BodyType_Weight.value_counts()

80kg)     606
75kg)     539
78kg)     496
70kg)     419
82kg)     365
77kg)     346
76kg)     344
79kg)     333
81kg)     327
83kg)     325
73kg)     316
74kg)     316
84kg)     291
72kg)     289
85kg)     286
68kg)     217
88kg)     195
71kg)     195
67kg)     190
86kg)     189
69kg)     184
65kg)     167
66kg)     167
60kg)     160
87kg)     152
62kg)     142
64kg)     138
90kg)     136
63kg)     112
61kg)     110
58kg)     109
89kg)     107
59kg)      86
57kg)      66
56kg)      66
55kg)      64
91kg)      63
92kg)      62
94kg)      56
93kg)      53
54kg)      52
95kg)      26
53kg)      25
52kg)      23
96kg)      20
51kg)      15
98kg)      14
97kg)      10
99kg)       7
100kg)      4
49kg)       4
102kg)      3
48kg)       2
103kg)      2
104kg)      1
105kg)      1
101kg)      1
47kg)       1
46kg)       1
50kg)       1
Name: BodyType_Weight, dtype: int64

In [42]:
futbin_data.BodyType_Weight = futbin_data.BodyType_Weight.str[:-3]

In [43]:
futbin_data.BodyType_Weight = pd.to_numeric(futbin_data.BodyType_Weight)

In [44]:
futbin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8997 entries, 0 to 22580
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    8997 non-null   object 
 1   Club                    8997 non-null   object 
 2   Nation                  8997 non-null   object 
 3   League                  8997 non-null   object 
 4   Rating                  8997 non-null   int64  
 5   Main_Position           8997 non-null   object 
 6   Alternate_Positions     8997 non-null   object 
 7   Card_Version            8997 non-null   object 
 8   Run_Style               8997 non-null   object 
 9   Price                   8997 non-null   float64
 10  Skills_Star             8997 non-null   int64  
 11  Weak_Foot_Star          8997 non-null   int64  
 12  Attack_Workrate         8997 non-null   object 
 13  Defense_Workrate        8997 non-null   object 
 14  Pace / Diving           8997 non-null  

In [45]:
futbin_data.head()

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight
0,Erling Haaland,Manchester City,Norway,Premier League,97,ST,CF,TOTY,Controlled Lengthy,1980000.0,4,5,H,M,96.0,98,80,91,56,97,"195cm | 6'5""",Unique (94kg),1645,518,2481,Male,195,Unique,94
1,Kylian Mbappé,Paris SG,France,Ligue 1 Uber Eats,97,ST,"CF,LW",TOTY,Controlled Explosive,6250000.0,5,4,H,L,99.0,96,88,97,47,88,"182cm | 6'0""",Unique (75kg),1651,515,2465,Male,182,Unique,75
2,Lionel Messi,Inter Miami CF,Argentina,MLS,97,CF,"CAM,RW,ST",TOTY,Mostly Explosive,2830000.0,5,4,M,L,91.0,96,97,98,40,77,"169cm | 5'7""",Messi (67kg),-4941,499,2430,Male,169,Messi,67
3,Pelé,EA FC ICONS,Brazil,Icons,97,CAM,"CF,ST",FUT Birthday Icon,Mostly Explosive,6400000.0,5,5,H,M,95.0,96,92,96,60,81,"173cm | 5'8""",Unique (70kg),272,520,2532,Male,173,Unique,70
4,Johan Cruyff,EA FC ICONS,Holland,Icons,97,CF,ST,Golazo Icon,Mostly Explosive,3550000.0,5,5,H,L,93.0,95,93,97,50,80,"180cm | 5'11""",Avg & Lean (71kg),159,508,2460,Male,180,Avg & Lean,71


Remove extracted columns

In [46]:
futbin_data = futbin_data.drop(['Height','BodyType'], 1)

In [47]:
futbin_data.Club.value_counts()

EA FC ICONS                247
HERO                       127
Paris SG                   106
FC Barcelona               105
Real Madrid                105
Manchester City            102
FC Bayern München           94
Arsenal                     90
Chelsea                     90
Liverpool                   90
OL                          80
Atlético de Madrid          75
Manchester Utd              75
Leverkusen                  69
Spurs                       66
Frankfurt                   61
Inter                       58
Sevilla FC                  53
Juventus                    52
Aston Villa                 52
RB Leipzig                  50
Milan                       49
VfL Wolfsburg               47
Athletic Club               47
Borussia Dortmund           46
Brighton                    46
TSG Hoffenheim              46
West Ham                    44
SC Freiburg                 42
Roma FC                     41
Real Sociedad               41
SL Benfica                  38
LOSC Lil

#### Club

It has many values, I'll create a new column which will have 1 if the club is FUT ICONS or HERO and 0 otherwise.

In [48]:
# Defining the map function
def icon_map(x):
    if x == 'EA FC ICONS':
        return 1
    elif x == 'HERO':
        return 1
    else:
        return 0

# Applying the function to the club column
futbin_data['Club_Hero'] = futbin_data['Club'].apply(icon_map)

In [49]:
futbin_data.Club_Hero.value_counts()

0    8623
1     374
Name: Club_Hero, dtype: int64

In [50]:
futbin_data = futbin_data.drop(['Club'], 1)

In [51]:
futbin_data.head()

,Name,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero
0,Erling Haaland,Norway,Premier League,97,ST,CF,TOTY,Controlled Lengthy,1980000.0,4,5,H,M,96.0,98,80,91,56,97,1645,518,2481,Male,195,Unique,94,0
1,Kylian Mbappé,France,Ligue 1 Uber Eats,97,ST,"CF,LW",TOTY,Controlled Explosive,6250000.0,5,4,H,L,99.0,96,88,97,47,88,1651,515,2465,Male,182,Unique,75,0
2,Lionel Messi,Argentina,MLS,97,CF,"CAM,RW,ST",TOTY,Mostly Explosive,2830000.0,5,4,M,L,91.0,96,97,98,40,77,-4941,499,2430,Male,169,Messi,67,0
3,Pelé,Brazil,Icons,97,CAM,"CF,ST",FUT Birthday Icon,Mostly Explosive,6400000.0,5,5,H,M,95.0,96,92,96,60,81,272,520,2532,Male,173,Unique,70,1
4,Johan Cruyff,Holland,Icons,97,CF,ST,Golazo Icon,Mostly Explosive,3550000.0,5,5,H,L,93.0,95,93,97,50,80,159,508,2460,Male,180,Avg & Lean,71,1


#### League

In [52]:
futbin_data.League.value_counts()

Premier League             688
LALIGA EA SPORTS           543
Serie A TIM                503
Bundesliga                 474
Ligue 1 Uber Eats          390
MLS                        298
NWSL                       289
Barclays WSL               260
Liga F                     249
Icons                      247
Sudamericana               232
EFL Championship           231
ROSHN Saudi League         227
GPFBL                      223
EFL League Two             221
EFL League One             217
Trendyol Süper Lig         204
Eredivisie                 202
D1 Arkema                  196
Liga Portugal              195
Bundesliga 2               180
Libertadores               172
3. Liga                    172
1A Pro League              169
PKO BP Ekstraklasa         155
LALIGA HYPERMOTION         149
CSL                        146
Allsvenskan                138
Serie BKT                  131
Eliteserien                130
3F Superliga               129
Ligue 2 BKT                129
LPF     

We have many leagues, I'll collect the 5 mayor european leagues, and some special cases (Worls Cup, Icons), and all of the other values will be other. After this simplification I'll create categorical columns from the values.

In [53]:
# Defining the map function
def special_club_map(x):
    if x == 'Icons':
        return 'Icons'
    elif x == 'LALIGA EA SPORTS':
        return 'LALIGA EA SPORTS'
    elif x == 'Premier League':
        return 'Premier League'
    elif x == 'MLS':
        return 'MLS'
    elif x == 'Serie A TIM':
        return 'Serie A TIM'
    elif x == 'Ligue 1 Uber Eats':
        return 'Ligue 1 Uber Eats'
    elif x == 'Bundesliga':
        return 'Bundesliga'
    else:
        return 'Other'

# Applying the function to the club column
futbin_data['League_Cat'] = futbin_data['League'].apply(special_club_map)

In [54]:
futbin_data.League_Cat.value_counts()

Other                5854
Premier League        688
LALIGA EA SPORTS      543
Serie A TIM           503
Bundesliga            474
Ligue 1 Uber Eats     390
MLS                   298
Icons                 247
Name: League_Cat, dtype: int64

In [55]:
futbin_data = futbin_data.drop(['League'], 1)

In [56]:
futbin_data.head()

,Name,Nation,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat
0,Erling Haaland,Norway,97,ST,CF,TOTY,Controlled Lengthy,1980000.0,4,5,H,M,96.0,98,80,91,56,97,1645,518,2481,Male,195,Unique,94,0,Premier League
1,Kylian Mbappé,France,97,ST,"CF,LW",TOTY,Controlled Explosive,6250000.0,5,4,H,L,99.0,96,88,97,47,88,1651,515,2465,Male,182,Unique,75,0,Ligue 1 Uber Eats
2,Lionel Messi,Argentina,97,CF,"CAM,RW,ST",TOTY,Mostly Explosive,2830000.0,5,4,M,L,91.0,96,97,98,40,77,-4941,499,2430,Male,169,Messi,67,0,MLS
3,Pelé,Brazil,97,CAM,"CF,ST",FUT Birthday Icon,Mostly Explosive,6400000.0,5,5,H,M,95.0,96,92,96,60,81,272,520,2532,Male,173,Unique,70,1,Icons
4,Johan Cruyff,Holland,97,CF,ST,Golazo Icon,Mostly Explosive,3550000.0,5,5,H,L,93.0,95,93,97,50,80,159,508,2460,Male,180,Avg & Lean,71,1,Icons


#### Nation

I'll check the nation column, but I have a feeling that I need to remove it, because it has probably a lot of values and it will be hard to extract information from it.

In [57]:
futbin_data.Nation.value_counts()

England                     841
Germany                     750
France                      673
Spain                       665
Argentina                   390
United States               386
Italy                       340
Brazil                      337
Holland                     305
Portugal                    237
Sweden                      190
Poland                      190
Denmark                     181
Norway                      173
Belgium                     168
Republic of Ireland         159
Switzerland                 139
Austria                     135
China PR                    124
Scotland                    120
Colombia                    117
Uruguay                     103
Korea Republic              100
Turkey                       97
Australia                    95
Romania                      91
Croatia                      91
Saudi Arabia                 87
Serbia                       79
Canada                       76
Czech Republic               62
Nigeria 

We have a few nations more than 1000 players so I decided that I will keep those, and will use other for the others. It is just a subjective decision, so later it can be modified, and maybe keep other nations as well.

One other derived direction could be to merge the countries by continents Europe, America, Asia, and so on...

In [58]:
# Defining the map function
def nation_keep_map(x):
    if x == 'England':
        return 'England'
    elif x == 'Germany':
        return 'Germany'
    elif x == 'Spain':
        return 'Spain'
    elif x == 'France':
        return 'France'
    elif x == 'Argentina':
        return 'Argentina'
    elif x == 'United States':
        return 'United States'
    elif x == 'Italy':
        return 'Italy'
    elif x == 'Brazil':
        return 'Brazil'
    elif x == 'Holland':
        return 'Holland'
    elif x == 'Portugal':
        return 'Portugal'
    else:
        return 'Other'

# Applying the function to the club column
futbin_data['Nation_Cat'] = futbin_data['Nation'].apply(nation_keep_map)

In [59]:
futbin_data.Nation_Cat.value_counts()

Other            4073
England           841
Germany           750
France            673
Spain             665
Argentina         390
United States     386
Italy             340
Brazil            337
Holland           305
Portugal          237
Name: Nation_Cat, dtype: int64

In [60]:
futbin_data = futbin_data.drop(['Nation'], 1)

In [61]:
futbin_data.head()

,Name,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat
0,Erling Haaland,97,ST,CF,TOTY,Controlled Lengthy,1980000.0,4,5,H,M,96.0,98,80,91,56,97,1645,518,2481,Male,195,Unique,94,0,Premier League,Other
1,Kylian Mbappé,97,ST,"CF,LW",TOTY,Controlled Explosive,6250000.0,5,4,H,L,99.0,96,88,97,47,88,1651,515,2465,Male,182,Unique,75,0,Ligue 1 Uber Eats,France
2,Lionel Messi,97,CF,"CAM,RW,ST",TOTY,Mostly Explosive,2830000.0,5,4,M,L,91.0,96,97,98,40,77,-4941,499,2430,Male,169,Messi,67,0,MLS,Argentina
3,Pelé,97,CAM,"CF,ST",FUT Birthday Icon,Mostly Explosive,6400000.0,5,5,H,M,95.0,96,92,96,60,81,272,520,2532,Male,173,Unique,70,1,Icons,Brazil
4,Johan Cruyff,97,CF,ST,Golazo Icon,Mostly Explosive,3550000.0,5,5,H,L,93.0,95,93,97,50,80,159,508,2460,Male,180,Avg & Lean,71,1,Icons,Holland


#### Card_Version

In [62]:
futbin_data.Card_Version.value_counts()

normal                  6928
IF                       445
Normal                   141
TOTS                     117
Icon                      93
Hero                      75
TOTS Moments              66
SIF                       66
FUT Birthday              65
Winter Wildcards          49
ShowDown SBC              43
TOTS Plus                 40
Trailblazers              37
Future Stars              33
FUT Birthday Icon         32
Fantasy FC                31
Winter Wildcard ICON      29
Dynasties                 28
FUT Centurions            27
Versus Fire               27
Versus Ice                26
Champions League          25
TOTY                      24
Radioactive               22
Thunderstruck             22
Flashback SBC             20
Golazo Hero               20
TOTS SBC                  20
UCL TOTGS                 17
Fantasy FC Hero           17
FC Pro                    16
Golazo Icon               16
TOTS Live                 16
Triple Threat             15
Thunderstruck 

too many options and the values are mixed, not just card versions (toty, fut hero, winter wildcards, etc) but positions (LB, RB, etc and these combinations as well)

I'll just remove this column for now.

In [63]:
futbin_data = futbin_data.drop(['Card_Version'], 1)

In [64]:
futbin_data.head()

,Name,Rating,Main_Position,Alternate_Positions,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat
0,Erling Haaland,97,ST,CF,Controlled Lengthy,1980000.0,4,5,H,M,96.0,98,80,91,56,97,1645,518,2481,Male,195,Unique,94,0,Premier League,Other
1,Kylian Mbappé,97,ST,"CF,LW",Controlled Explosive,6250000.0,5,4,H,L,99.0,96,88,97,47,88,1651,515,2465,Male,182,Unique,75,0,Ligue 1 Uber Eats,France
2,Lionel Messi,97,CF,"CAM,RW,ST",Mostly Explosive,2830000.0,5,4,M,L,91.0,96,97,98,40,77,-4941,499,2430,Male,169,Messi,67,0,MLS,Argentina
3,Pelé,97,CAM,"CF,ST",Mostly Explosive,6400000.0,5,5,H,M,95.0,96,92,96,60,81,272,520,2532,Male,173,Unique,70,1,Icons,Brazil
4,Johan Cruyff,97,CF,ST,Mostly Explosive,3550000.0,5,5,H,L,93.0,95,93,97,50,80,159,508,2460,Male,180,Avg & Lean,71,1,Icons,Holland


#### Positions

In [65]:
futbin_data.Main_Position.value_counts()

CB     2782
GK     2125
CM     1099
ST      696
CAM     496
CDM     480
RB      238
LB      214
RW      191
LW      179
LM      168
RM      164
CF       93
RWB      37
LWB      35
Name: Main_Position, dtype: int64

would be perfect for categorical columns, will convert it at the last step.

In [66]:
futbin_data.Alternate_Positions.value_counts()

0             5851
CF             503
CM             288
CDM            199
LWB            109
RWB            108
RM              94
RB              69
CAM             67
LM              64
LM,RW           59
RM,LW           57
LW              55
LWB,LM          53
RW              52
CDM,CAM         52
CF,LW           52
ST              51
LB              45
CAM,CF          40
CF,RW           34
RM,ST           33
LM,CF           32
RWB,LB          29
RWB,CB          28
RW,ST           26
CM,LM           26
LM,ST           26
CAM,ST          25
CAM,LW          24
ST,LW           23
RWB,RM          23
LM,RW,ST        21
RM,CAM          21
RB,RM           21
LM,CAM          21
LB,LM           20
RM,CF           20
CB,CM           19
RM,CAM,LW       18
RB,LWB          18
LM,CF,LW        18
CAM,RW          18
CF,ST           16
CB              16
LM,CF,ST        15
RM,CM           15
RM,ST,LW        15
LWB,LW          15
CB,LWB          14
LM,RW,LW        14
CM,RW           14
RW,LW       

In [67]:
futbin_data['Alt_Pos_List'] = futbin_data['Alternate_Positions'].str.split(',')

In [68]:
futbin_data['Alt_Pos_List'].head()

0             [CF]
1         [CF, LW]
2    [CAM, RW, ST]
3         [CF, ST]
4             [ST]
Name: Alt_Pos_List, dtype: object

In [69]:
#futbin_data[['Alt_Pos_1', 'Alt_Pos_2', 'Alt_Pos_3']] = futbin_data['Alternate_Positions'].str.split(',', 1, expand=True)

In [70]:
futbin_data = futbin_data.join(futbin_data['Alternate_Positions'].str.split(',', expand=True).rename(columns={0:'Alt_Pos_1', 1:'Alt_Pos_2', 2:'Alt_Pos_3'}))

In [71]:
futbin_data.head()

,Name,Rating,Main_Position,Alternate_Positions,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_List,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3
0,Erling Haaland,97,ST,CF,Controlled Lengthy,1980000.0,4,5,H,M,96.0,98,80,91,56,97,1645,518,2481,Male,195,Unique,94,0,Premier League,Other,[CF],CF,None,None
1,Kylian Mbappé,97,ST,"CF,LW",Controlled Explosive,6250000.0,5,4,H,L,99.0,96,88,97,47,88,1651,515,2465,Male,182,Unique,75,0,Ligue 1 Uber Eats,France,"[CF, LW]",CF,LW,None
2,Lionel Messi,97,CF,"CAM,RW,ST",Mostly Explosive,2830000.0,5,4,M,L,91.0,96,97,98,40,77,-4941,499,2430,Male,169,Messi,67,0,MLS,Argentina,"[CAM, RW, ST]",CAM,RW,ST
3,Pelé,97,CAM,"CF,ST",Mostly Explosive,6400000.0,5,5,H,M,95.0,96,92,96,60,81,272,520,2532,Male,173,Unique,70,1,Icons,Brazil,"[CF, ST]",CF,ST,None
4,Johan Cruyff,97,CF,ST,Mostly Explosive,3550000.0,5,5,H,L,93.0,95,93,97,50,80,159,508,2460,Male,180,Avg & Lean,71,1,Icons,Holland,[ST],ST,None,None


In [72]:
futbin_data['Alt_Pos_Count'] = futbin_data['Alt_Pos_List'].str.len()

In [73]:
futbin_data = futbin_data.drop(['Alternate_Positions'], 1)

In [74]:
futbin_data = futbin_data.drop(['Alt_Pos_List'], 1)

In [75]:
futbin_data.head()

,Name,Rating,Main_Position,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count
0,Erling Haaland,97,ST,Controlled Lengthy,1980000.0,4,5,H,M,96.0,98,80,91,56,97,1645,518,2481,Male,195,Unique,94,0,Premier League,Other,CF,None,None,1
1,Kylian Mbappé,97,ST,Controlled Explosive,6250000.0,5,4,H,L,99.0,96,88,97,47,88,1651,515,2465,Male,182,Unique,75,0,Ligue 1 Uber Eats,France,CF,LW,None,2
2,Lionel Messi,97,CF,Mostly Explosive,2830000.0,5,4,M,L,91.0,96,97,98,40,77,-4941,499,2430,Male,169,Messi,67,0,MLS,Argentina,CAM,RW,ST,3
3,Pelé,97,CAM,Mostly Explosive,6400000.0,5,5,H,M,95.0,96,92,96,60,81,272,520,2532,Male,173,Unique,70,1,Icons,Brazil,CF,ST,None,2
4,Johan Cruyff,97,CF,Mostly Explosive,3550000.0,5,5,H,L,93.0,95,93,97,50,80,159,508,2460,Male,180,Avg & Lean,71,1,Icons,Holland,ST,None,None,1


In [76]:
futbin_data.tail()

,Name,Rating,Main_Position,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count
22573,Alexane Gery,56,GK,Controlled,200.0,1,2,M,M,61.0,53,50,59,51,50,1,324,684,Female,168,Short & Lean,60,0,Other,France,0,None,None,1
22574,Tereza Fuchsová,54,GK,Controlled,200.0,1,4,M,M,55.0,51,50,54,33,54,-115,297,701,Female,174,Avg & Normal,66,0,Other,Other,0,None,None,1
22577,Aleigh Gambone,52,CM,Controlled,200.0,4,4,H,M,60.0,55,53,56,49,52,-1,325,1580,Female,170,Short & Lean,60,0,Other,United States,0,None,None,1
22578,Joy Lysser,52,CM,Controlled,200.0,2,3,M,M,61.0,52,46,54,53,59,7,325,1487,Female,170,Short & Lean,62,0,Other,Other,0,None,None,1
22580,Soraya Wulff,52,CB,Controlled,8900.0,2,3,L,H,59.0,42,51,52,51,56,2,311,1447,Female,174,Avg & Stocky,72,0,Other,Other,0,None,None,1


In [77]:
futbin_data.Alt_Pos_1.replace('0', np.nan, inplace=True)

In [78]:
futbin_data.Alt_Pos_2.fillna(value=np.nan, inplace=True)

In [79]:
futbin_data.Alt_Pos_3.fillna(value=np.nan, inplace=True)

Need to update Alt_Pos_Count to 0 if the Alt_Pos_1 is NaN, because originally, the field conatins 0 instead of null value.

In [80]:
#futbin_data['Alt_Pos_Count'] = futbin_data.apply(lambda x: 0 if x['Alt_Pos_1']=='NaN' else x['Alt_Pos_Count'], axis=1)
futbin_data.Alt_Pos_Count = np.where(futbin_data.Alt_Pos_1.isna(), 0, futbin_data.Alt_Pos_Count)

In [81]:
futbin_data.Alt_Pos_1.info()

<class 'pandas.core.series.Series'>
Int64Index: 8997 entries, 0 to 22580
Series name: Alt_Pos_1
Non-Null Count  Dtype 
--------------  ----- 
3146 non-null   object
dtypes: object(1)
memory usage: 398.6+ KB


In [82]:
futbin_data[futbin_data.Alt_Pos_1.isna()].tail()

,Name,Rating,Main_Position,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count
22573,Alexane Gery,56,GK,Controlled,200.0,1,2,M,M,61.0,53,50,59,51,50,1,324,684,Female,168,Short & Lean,60,0,Other,France,NaN,NaN,NaN,0
22574,Tereza Fuchsová,54,GK,Controlled,200.0,1,4,M,M,55.0,51,50,54,33,54,-115,297,701,Female,174,Avg & Normal,66,0,Other,Other,NaN,NaN,NaN,0
22577,Aleigh Gambone,52,CM,Controlled,200.0,4,4,H,M,60.0,55,53,56,49,52,-1,325,1580,Female,170,Short & Lean,60,0,Other,United States,NaN,NaN,NaN,0
22578,Joy Lysser,52,CM,Controlled,200.0,2,3,M,M,61.0,52,46,54,53,59,7,325,1487,Female,170,Short & Lean,62,0,Other,Other,NaN,NaN,NaN,0
22580,Soraya Wulff,52,CB,Controlled,8900.0,2,3,L,H,59.0,42,51,52,51,56,2,311,1447,Female,174,Avg & Stocky,72,0,Other,Other,NaN,NaN,NaN,0


In [83]:
futbin_data.tail()

,Name,Rating,Main_Position,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count
22573,Alexane Gery,56,GK,Controlled,200.0,1,2,M,M,61.0,53,50,59,51,50,1,324,684,Female,168,Short & Lean,60,0,Other,France,NaN,NaN,NaN,0
22574,Tereza Fuchsová,54,GK,Controlled,200.0,1,4,M,M,55.0,51,50,54,33,54,-115,297,701,Female,174,Avg & Normal,66,0,Other,Other,NaN,NaN,NaN,0
22577,Aleigh Gambone,52,CM,Controlled,200.0,4,4,H,M,60.0,55,53,56,49,52,-1,325,1580,Female,170,Short & Lean,60,0,Other,United States,NaN,NaN,NaN,0
22578,Joy Lysser,52,CM,Controlled,200.0,2,3,M,M,61.0,52,46,54,53,59,7,325,1487,Female,170,Short & Lean,62,0,Other,Other,NaN,NaN,NaN,0
22580,Soraya Wulff,52,CB,Controlled,8900.0,2,3,L,H,59.0,42,51,52,51,56,2,311,1447,Female,174,Avg & Stocky,72,0,Other,Other,NaN,NaN,NaN,0


### 3. Categorical columns

Main_Position

Run_Style

Attack_Workrate

Defense_Workrate

BodyType_Text

League_Cat

Nation_Cat

======================================================

Alt_Pos_1

Alt_Pos_2

Alt_Pos_3

First I'll handle the "normal" cat columns, skipping the from alt pos, because those could have emty fields.

In [84]:
# Creating a dummy variable for some of the categorical variables and dropping the first one.
dummy_fields = pd.get_dummies(futbin_data[['Main_Position', 'Run_Style', 'Attack_Workrate', 'Defense_Workrate', 
                                           'BodyType_Text', 'League_Cat', 'Nation_Cat']], drop_first=True)

# Adding the results to the master dataframe
futbin_data = pd.concat([futbin_data, dummy_fields], axis=1)

In [85]:
futbin_data.head()

,Name,Rating,Main_Position,Run_Style,Price,Skills_Star,Weak_Foot_Star,Attack_Workrate,Defense_Workrate,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Text,BodyType_Weight,Club_Hero,League_Cat,Nation_Cat,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Controlled Explosive,Run_Style_Controlled Lengthy,Run_Style_Explosive,Run_Style_Lengthy,Run_Style_Mostly Explosive,Run_Style_Mostly Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_Avg & Normal,BodyType_Text_Avg & Stocky,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_Messi,BodyType_Text_Neymar,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Normal,BodyType_Text_Short & Stocky,BodyType_Text_Tall & Lean,BodyType_Text_Tall & Normal,BodyType_Text_Tall & Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LALIGA EA SPORTS,League_Cat_Ligue 1 Uber Eats,League_Cat_MLS,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_Brazil,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Holland,Nation_Cat_Italy,Nation_Cat_Other,Nation_Cat_Portugal,Nation_Cat_Spain,Nation_Cat_United States
0,Erling Haaland,97,ST,Controlled Lengthy,1980000.0,4,5,H,M,96.0,98,80,91,56,97,1645,518,2481,Male,195,Unique,94,0,Premier League,Other,CF,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,Kylian Mbappé,97,ST,Controlled Explosive,6250000.0,5,4,H,L,99.0,96,88,97,47,88,1651,515,2465,Male,182,Unique,75,0,Ligue 1 Uber Eats,France,CF,LW,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Lionel Messi,97,CF,Mostly Explosive,2830000.0,5,4,M,L,91.0,96,97,98,40,77,-4941,499,2430,Male,169,Messi,67,0,MLS,Argentina,CAM,RW,ST,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Pelé,97,CAM,Mostly Explosive,6400000.0,5,5,H,M,95.0,96,92,96,60,81,272,520,2532,Male,173,Unique,70,1,Icons,Brazil,CF,ST,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Johan Cruyff,97,CF,Mostly Explosive,3550000.0,5,5,H,L,93.0,95,93,97,50,80,159,508,2460,Male,180,Avg & Lean,71,1,Icons,Holland,ST,NaN,NaN,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [86]:
# dropping the repeated variables
futbin_data = futbin_data.drop(['Main_Position', 'Run_Style', 'Attack_Workrate', 
                        'Defense_Workrate', 'BodyType_Text', 'League_Cat', 'Nation_Cat'], 1)

In [87]:
futbin_data.head()

,Name,Rating,Price,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Weight,Club_Hero,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Controlled Explosive,Run_Style_Controlled Lengthy,Run_Style_Explosive,Run_Style_Lengthy,Run_Style_Mostly Explosive,Run_Style_Mostly Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_Avg & Normal,BodyType_Text_Avg & Stocky,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_Messi,BodyType_Text_Neymar,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Normal,BodyType_Text_Short & Stocky,BodyType_Text_Tall & Lean,BodyType_Text_Tall & Normal,BodyType_Text_Tall & Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LALIGA EA SPORTS,League_Cat_Ligue 1 Uber Eats,League_Cat_MLS,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_Brazil,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Holland,Nation_Cat_Italy,Nation_Cat_Other,Nation_Cat_Portugal,Nation_Cat_Spain,Nation_Cat_United States
0,Erling Haaland,97,1980000.0,4,5,96.0,98,80,91,56,97,1645,518,2481,Male,195,94,0,CF,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,Kylian Mbappé,97,6250000.0,5,4,99.0,96,88,97,47,88,1651,515,2465,Male,182,75,0,CF,LW,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Lionel Messi,97,2830000.0,5,4,91.0,96,97,98,40,77,-4941,499,2430,Male,169,67,0,CAM,RW,ST,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Pelé,97,6400000.0,5,5,95.0,96,92,96,60,81,272,520,2532,Male,173,70,1,CF,ST,NaN,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Johan Cruyff,97,3550000.0,5,5,93.0,95,93,97,50,80,159,508,2460,Male,180,71,1,ST,NaN,NaN,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


#### Alt positins filling with missing value, and use it as a category

##### Alt_Pos_1

In [88]:
futbin_data.Alt_Pos_1.isnull().sum()

5851

In [89]:
futbin_data.Alt_Pos_1.fillna('missing', inplace=True)

In [90]:
futbin_data.Alt_Pos_1.isnull().sum()

0

In [91]:
futbin_data.Alt_Pos_1.value_counts()

missing    5851
CF          627
CM          393
RM          377
LM          310
CDM         282
RWB         259
CAM         192
LWB         188
RB          138
RW           99
LB           91
ST           74
CB           61
LW           55
Name: Alt_Pos_1, dtype: int64

##### Alt_Pos_2

In [92]:
futbin_data.Alt_Pos_2.isnull().sum()

7571

In [93]:
futbin_data.Alt_Pos_2.fillna('missing', inplace=True)

In [94]:
futbin_data.Alt_Pos_2.isnull().sum()

0

In [95]:
futbin_data.Alt_Pos_2.value_counts()

missing    7571
RW          221
LW          214
ST          177
CAM         177
CF          164
LM          154
CM           78
RM           65
LB           62
LWB          45
CB           40
CDM          18
RB           11
Name: Alt_Pos_2, dtype: int64

##### Alt_Pos_3

In [96]:
futbin_data.Alt_Pos_3.isnull().sum()

8621

In [97]:
futbin_data.Alt_Pos_3.fillna('missing', inplace=True)

In [98]:
futbin_data.Alt_Pos_3.isnull().sum()

0

In [99]:
futbin_data.Alt_Pos_3.value_counts()

missing    8621
LW          146
ST           77
RW           65
CF           25
LM           20
LWB          10
RM            9
CM            8
CAM           7
CDM           5
LB            4
Name: Alt_Pos_3, dtype: int64

In [100]:
futbin_data.head()

,Name,Rating,Price,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Weight,Club_Hero,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Controlled Explosive,Run_Style_Controlled Lengthy,Run_Style_Explosive,Run_Style_Lengthy,Run_Style_Mostly Explosive,Run_Style_Mostly Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_Avg & Normal,BodyType_Text_Avg & Stocky,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_Messi,BodyType_Text_Neymar,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Normal,BodyType_Text_Short & Stocky,BodyType_Text_Tall & Lean,BodyType_Text_Tall & Normal,BodyType_Text_Tall & Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LALIGA EA SPORTS,League_Cat_Ligue 1 Uber Eats,League_Cat_MLS,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_Brazil,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Holland,Nation_Cat_Italy,Nation_Cat_Other,Nation_Cat_Portugal,Nation_Cat_Spain,Nation_Cat_United States
0,Erling Haaland,97,1980000.0,4,5,96.0,98,80,91,56,97,1645,518,2481,Male,195,94,0,CF,missing,missing,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,Kylian Mbappé,97,6250000.0,5,4,99.0,96,88,97,47,88,1651,515,2465,Male,182,75,0,CF,LW,missing,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Lionel Messi,97,2830000.0,5,4,91.0,96,97,98,40,77,-4941,499,2430,Male,169,67,0,CAM,RW,ST,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Pelé,97,6400000.0,5,5,95.0,96,92,96,60,81,272,520,2532,Male,173,70,1,CF,ST,missing,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Johan Cruyff,97,3550000.0,5,5,93.0,95,93,97,50,80,159,508,2460,Male,180,71,1,ST,missing,missing,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


#### Create categorical columns for the alt pos columns

In [101]:
# Creating a dummy variable for the alt_pos categorical variables and dropping the first one.
dummy_altpos = pd.get_dummies(futbin_data[['Alt_Pos_1', 'Alt_Pos_2', 'Alt_Pos_3']], drop_first=True)

# Adding the results to the master dataframe
futbin_data = pd.concat([futbin_data, dummy_altpos], axis=1)

In [102]:
futbin_data.head()

,Name,Rating,Price,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Weight,Club_Hero,Alt_Pos_1,Alt_Pos_2,Alt_Pos_3,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Controlled Explosive,Run_Style_Controlled Lengthy,Run_Style_Explosive,Run_Style_Lengthy,Run_Style_Mostly Explosive,Run_Style_Mostly Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_Avg & Normal,BodyType_Text_Avg & Stocky,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_Messi,BodyType_Text_Neymar,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Normal,BodyType_Text_Short & Stocky,BodyType_Text_Tall & Lean,BodyType_Text_Tall & Normal,BodyType_Text_Tall & Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LALIGA EA SPORTS,League_Cat_Ligue 1 Uber Eats,League_Cat_MLS,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_Brazil,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Holland,Nation_Cat_Italy,Nation_Cat_Other,Nation_Cat_Portugal,Nation_Cat_Spain,Nation_Cat_United States,Alt_Pos_1_CB,Alt_Pos_1_CDM,Alt_Pos_1_CF,Alt_Pos_1_CM,Alt_Pos_1_LB,Alt_Pos_1_LM,Alt_Pos_1_LW,Alt_Pos_1_LWB,Alt_Pos_1_RB,Alt_Pos_1_RM,Alt_Pos_1_RW,Alt_Pos_1_RWB,Alt_Pos_1_ST,Alt_Pos_1_missing,Alt_Pos_2_CB,Alt_Pos_2_CDM,Alt_Pos_2_CF,Alt_Pos_2_CM,Alt_Pos_2_LB,Alt_Pos_2_LM,Alt_Pos_2_LW,Alt_Pos_2_LWB,Alt_Pos_2_RB,Alt_Pos_2_RM,Alt_Pos_2_RW,Alt_Pos_2_ST,Alt_Pos_2_missing,Alt_Pos_3_CDM,Alt_Pos_3_CF,Alt_Pos_3_CM,Alt_Pos_3_LB,Alt_Pos_3_LM,Alt_Pos_3_LW,Alt_Pos_3_LWB,Alt_Pos_3_RM,Alt_Pos_3_RW,Alt_Pos_3_ST,Alt_Pos_3_missing
0,Erling Haaland,97,1980000.0,4,5,96.0,98,80,91,56,97,1645,518,2481,Male,195,94,0,CF,missing,missing,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,Kylian Mbappé,97,6250000.0,5,4,99.0,96,88,97,47,88,1651,515,2465,Male,182,75,0,CF,LW,missing,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Lionel Messi,97,2830000.0,5,4,91.0,96,97,98,40,77,-4941,499,2430,Male,169,67,0,CAM,RW,ST,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Pelé,97,6400000.0,5,5,95.0,96,92,96,60,81,272,520,2532,Male,173,70,1,CF,ST,missing,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,Johan Cruyff,97,3550000.0,5,5,93.0,95,93,97,50,80,159,508,2460,Male,180,71,1,ST,missing,missing,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [103]:
# dropping the repeated variables
futbin_data = futbin_data.drop(['Alt_Pos_1', 'Alt_Pos_2', 'Alt_Pos_3'], 1)

In [104]:
futbin_data.head()

,Name,Rating,Price,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Weight,Club_Hero,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Controlled Explosive,Run_Style_Controlled Lengthy,Run_Style_Explosive,Run_Style_Lengthy,Run_Style_Mostly Explosive,Run_Style_Mostly Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_Avg & Normal,BodyType_Text_Avg & Stocky,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_Messi,BodyType_Text_Neymar,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Normal,BodyType_Text_Short & Stocky,BodyType_Text_Tall & Lean,BodyType_Text_Tall & Normal,BodyType_Text_Tall & Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LALIGA EA SPORTS,League_Cat_Ligue 1 Uber Eats,League_Cat_MLS,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_Brazil,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Holland,Nation_Cat_Italy,Nation_Cat_Other,Nation_Cat_Portugal,Nation_Cat_Spain,Nation_Cat_United States,Alt_Pos_1_CB,Alt_Pos_1_CDM,Alt_Pos_1_CF,Alt_Pos_1_CM,Alt_Pos_1_LB,Alt_Pos_1_LM,Alt_Pos_1_LW,Alt_Pos_1_LWB,Alt_Pos_1_RB,Alt_Pos_1_RM,Alt_Pos_1_RW,Alt_Pos_1_RWB,Alt_Pos_1_ST,Alt_Pos_1_missing,Alt_Pos_2_CB,Alt_Pos_2_CDM,Alt_Pos_2_CF,Alt_Pos_2_CM,Alt_Pos_2_LB,Alt_Pos_2_LM,Alt_Pos_2_LW,Alt_Pos_2_LWB,Alt_Pos_2_RB,Alt_Pos_2_RM,Alt_Pos_2_RW,Alt_Pos_2_ST,Alt_Pos_2_missing,Alt_Pos_3_CDM,Alt_Pos_3_CF,Alt_Pos_3_CM,Alt_Pos_3_LB,Alt_Pos_3_LM,Alt_Pos_3_LW,Alt_Pos_3_LWB,Alt_Pos_3_RM,Alt_Pos_3_RW,Alt_Pos_3_ST,Alt_Pos_3_missing
0,Erling Haaland,97,1980000.0,4,5,96.0,98,80,91,56,97,1645,518,2481,Male,195,94,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,Kylian Mbappé,97,6250000.0,5,4,99.0,96,88,97,47,88,1651,515,2465,Male,182,75,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Lionel Messi,97,2830000.0,5,4,91.0,96,97,98,40,77,-4941,499,2430,Male,169,67,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Pelé,97,6400000.0,5,5,95.0,96,92,96,60,81,272,520,2532,Male,173,70,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,Johan Cruyff,97,3550000.0,5,5,93.0,95,93,97,50,80,159,508,2460,Male,180,71,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


#### Gender column handling

In [106]:
# Creating a dummy variable for some of the categorical variables and dropping the first one.
dummy_fields = pd.get_dummies(futbin_data[['Gender']], drop_first=True)

# Adding the results to the master dataframe
futbin_data = pd.concat([futbin_data, dummy_fields], axis=1)

In [107]:
futbin_data.head()

,Name,Rating,Price,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Gender,Height_in_cm,BodyType_Weight,Club_Hero,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Controlled Explosive,Run_Style_Controlled Lengthy,Run_Style_Explosive,Run_Style_Lengthy,Run_Style_Mostly Explosive,Run_Style_Mostly Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_Avg & Normal,BodyType_Text_Avg & Stocky,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_Messi,BodyType_Text_Neymar,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Normal,BodyType_Text_Short & Stocky,BodyType_Text_Tall & Lean,BodyType_Text_Tall & Normal,BodyType_Text_Tall & Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LALIGA EA SPORTS,League_Cat_Ligue 1 Uber Eats,League_Cat_MLS,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_Brazil,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Holland,Nation_Cat_Italy,Nation_Cat_Other,Nation_Cat_Portugal,Nation_Cat_Spain,Nation_Cat_United States,Alt_Pos_1_CB,Alt_Pos_1_CDM,Alt_Pos_1_CF,Alt_Pos_1_CM,Alt_Pos_1_LB,Alt_Pos_1_LM,Alt_Pos_1_LW,Alt_Pos_1_LWB,Alt_Pos_1_RB,Alt_Pos_1_RM,Alt_Pos_1_RW,Alt_Pos_1_RWB,Alt_Pos_1_ST,Alt_Pos_1_missing,Alt_Pos_2_CB,Alt_Pos_2_CDM,Alt_Pos_2_CF,Alt_Pos_2_CM,Alt_Pos_2_LB,Alt_Pos_2_LM,Alt_Pos_2_LW,Alt_Pos_2_LWB,Alt_Pos_2_RB,Alt_Pos_2_RM,Alt_Pos_2_RW,Alt_Pos_2_ST,Alt_Pos_2_missing,Alt_Pos_3_CDM,Alt_Pos_3_CF,Alt_Pos_3_CM,Alt_Pos_3_LB,Alt_Pos_3_LM,Alt_Pos_3_LW,Alt_Pos_3_LWB,Alt_Pos_3_RM,Alt_Pos_3_RW,Alt_Pos_3_ST,Alt_Pos_3_missing,Gender_Male
0,Erling Haaland,97,1980000.0,4,5,96.0,98,80,91,56,97,1645,518,2481,Male,195,94,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1
1,Kylian Mbappé,97,6250000.0,5,4,99.0,96,88,97,47,88,1651,515,2465,Male,182,75,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2,Lionel Messi,97,2830000.0,5,4,91.0,96,97,98,40,77,-4941,499,2430,Male,169,67,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,Pelé,97,6400000.0,5,5,95.0,96,92,96,60,81,272,520,2532,Male,173,70,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
4,Johan Cruyff,97,3550000.0,5,5,93.0,95,93,97,50,80,159,508,2460,Male,180,71,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1


In [108]:
# dropping the repeated variables
futbin_data = futbin_data.drop(['Gender'], 1)

In [109]:
futbin_data.head()

,Name,Rating,Price,Skills_Star,Weak_Foot_Star,Pace / Diving,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Popularity,Base_Stats,Ingame_Stats,Height_in_cm,BodyType_Weight,Club_Hero,Alt_Pos_Count,Main_Position_CB,Main_Position_CDM,Main_Position_CF,Main_Position_CM,Main_Position_GK,Main_Position_LB,Main_Position_LM,Main_Position_LW,Main_Position_LWB,Main_Position_RB,Main_Position_RM,Main_Position_RW,Main_Position_RWB,Main_Position_ST,Run_Style_Controlled Explosive,Run_Style_Controlled Lengthy,Run_Style_Explosive,Run_Style_Lengthy,Run_Style_Mostly Explosive,Run_Style_Mostly Lengthy,Attack_Workrate_L,Attack_Workrate_M,Defense_Workrate_L,Defense_Workrate_M,BodyType_Text_Avg & Normal,BodyType_Text_Avg & Stocky,BodyType_Text_CR7,BodyType_Text_Courtois,BodyType_Text_Messi,BodyType_Text_Neymar,BodyType_Text_R9,BodyType_Text_Ronaldinho,BodyType_Text_Salah,BodyType_Text_Shaqiri,BodyType_Text_Short & Lean,BodyType_Text_Short & Normal,BodyType_Text_Short & Stocky,BodyType_Text_Tall & Lean,BodyType_Text_Tall & Normal,BodyType_Text_Tall & Stocky,BodyType_Text_Unique,League_Cat_Icons,League_Cat_LALIGA EA SPORTS,League_Cat_Ligue 1 Uber Eats,League_Cat_MLS,League_Cat_Other,League_Cat_Premier League,League_Cat_Serie A TIM,Nation_Cat_Brazil,Nation_Cat_England,Nation_Cat_France,Nation_Cat_Germany,Nation_Cat_Holland,Nation_Cat_Italy,Nation_Cat_Other,Nation_Cat_Portugal,Nation_Cat_Spain,Nation_Cat_United States,Alt_Pos_1_CB,Alt_Pos_1_CDM,Alt_Pos_1_CF,Alt_Pos_1_CM,Alt_Pos_1_LB,Alt_Pos_1_LM,Alt_Pos_1_LW,Alt_Pos_1_LWB,Alt_Pos_1_RB,Alt_Pos_1_RM,Alt_Pos_1_RW,Alt_Pos_1_RWB,Alt_Pos_1_ST,Alt_Pos_1_missing,Alt_Pos_2_CB,Alt_Pos_2_CDM,Alt_Pos_2_CF,Alt_Pos_2_CM,Alt_Pos_2_LB,Alt_Pos_2_LM,Alt_Pos_2_LW,Alt_Pos_2_LWB,Alt_Pos_2_RB,Alt_Pos_2_RM,Alt_Pos_2_RW,Alt_Pos_2_ST,Alt_Pos_2_missing,Alt_Pos_3_CDM,Alt_Pos_3_CF,Alt_Pos_3_CM,Alt_Pos_3_LB,Alt_Pos_3_LM,Alt_Pos_3_LW,Alt_Pos_3_LWB,Alt_Pos_3_RM,Alt_Pos_3_RW,Alt_Pos_3_ST,Alt_Pos_3_missing,Gender_Male
0,Erling Haaland,97,1980000.0,4,5,96.0,98,80,91,56,97,1645,518,2481,195,94,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1
1,Kylian Mbappé,97,6250000.0,5,4,99.0,96,88,97,47,88,1651,515,2465,182,75,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2,Lionel Messi,97,2830000.0,5,4,91.0,96,97,98,40,77,-4941,499,2430,169,67,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,Pelé,97,6400000.0,5,5,95.0,96,92,96,60,81,272,520,2532,173,70,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
4,Johan Cruyff,97,3550000.0,5,5,93.0,95,93,97,50,80,159,508,2460,180,71,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1


In [112]:
futbin_data.Gender_Male.value_counts()

1    7695
0    1302
Name: Gender_Male, dtype: int64

In EA FC 24, Female players were introduced, there are 1302 female players next to the 76

### 4. Export final dataset

In [105]:
csv_futbin = futbin_data.to_csv('futbin24.csv', index = False)
print('\nCSV String:\n', csv_futbin)


CSV String:
 None
